In [2]:
import sys
sys.path.append('../Code')
from utils import *
from beakerx import *
from beakerx.object import beakerx
import statsmodels.regression.linear_model as sm

# Replicating the PRF

My conjecture is that the PRF has not done a good job of predicting returns in the past 10 years. To test this, I will use the strongest version of the PRF estimator found in the paper -- in particular I am going to use the price moving average (price minus 3y moving average of price) of the top 500 stocks in each period to predict one month ahead returns. For simplicity, in each period I just use the In this notebook you will be able to see

1. The factor that's extracted month by month
2. The time series of the returns over time
3. A rough estimate of how the quality of the forecasts have changed over time.

For the data, I use the CRSP tape going back to 1929 to June 2018 as well as the CRSP value weighted returns.

In [8]:
forecast_frame = pd.read_hdf('../Output/forecasts_3prf.h5')
date_sequence = forecast_frame['datadate']

## Visualizing the Factor

In [9]:
plot = TimePlot(title = 'Return Factor', legendLayout=LegendLayout.HORIZONTAL,\
                      legendPosition=LegendPosition(position=LegendPosition.Position.TOP),\
                    initWidth = 1000)
plot.add(Line(displayName = 'Return Factor (F)', \
              x = forecast_frame['datadate'],\
              y = forecast_frame['Factor']))

## Trying to replicate their particular plot

In [10]:
ROLL_WINDOW = int(40) # Number of years for rolling window
MIN_WINDOW = int(10) # Number of years for minimum window

In [11]:
from copy import copy

def oos_by_split_date(dataframe, split_date):
    before = dataframe.loc[dataframe['datadate'] < split_date, :]    
    after = copy(dataframe.loc[dataframe['datadate'] >= split_date, :])
    
    if (before.shape[0] < MIN_WINDOW * 12) or (after.shape[0] < MIN_WINDOW * 12):
        return np.nan
    
    sample_mean_before_date = before['Next Month Return'].mean()
    factor_coef = sm.OLS(before['Next Month Return'], before[['Intercept', 'Factor']]).fit()

    pred_3prf = factor_coef.predict(after[['Intercept', 'Factor']])
    after.loc[:, '3PRF'] = pred_3prf
    after.loc[:, 'Mean'] = sample_mean_before_date
    mse_3prf = ((after.loc[:, '3PRF'] - after.loc[:, 'Next Month Return']) ** 2).mean()
    mse_sample_mean = ((after.loc[:, 'Mean'] - after.loc[:, 'Next Month Return']) ** 2).mean()
    oos_r2 = 1 - mse_3prf / mse_sample_mean
    return oos_r2

In [12]:
data_used_in_paper = forecast_frame.loc[forecast_frame['datadate'] <= '2010-12-31']

with Pool(4) as p:
    forecast_frame['OOS By Split Date (Paper Sample)'] = p.map(lambda t: oos_by_split_date(data_used_in_paper, t), date_sequence)
    forecast_frame['OOS By Split Date (2018 Sample)'] = p.map(lambda t: oos_by_split_date(forecast_frame, t), date_sequence)

In [14]:
forecast_frame.head()

In [15]:
plot = TimePlot(title = 'OOS R2 (Direct Replication of Plot)', legendLayout=LegendLayout.HORIZONTAL,\
                      legendPosition=LegendPosition(position=LegendPosition.Position.TOP),\
                    initWidth = 1000)
plot.add(Line(displayName = 'Through 2010', x = forecast_frame['datadate'], y = forecast_frame['OOS By Split Date (Paper Sample)']))
plot.add(Line(displayName = 'Through 2018', x = forecast_frame['datadate'], y = forecast_frame['OOS By Split Date (2018 Sample)']))

## A More Natural Recursive Updating Procedure

In [16]:
# Now do the expanding and rolling forecasts

def calculate_forecast_from_extracted_factor(data):    
    yesterday = data[:-1]
    today = data.tail(1)
    reg = sm.OLS(yesterday['Next Month Return'], yesterday[['Factor', 'Intercept']]).fit()
    return reg.predict(today[['Factor', 'Intercept']])[0]

def rolling_forecast_on_date(dataframe, target_date, window, min_periods): 
    valid_dates = dataframe.loc[(dataframe['datadate'] <= target_date) & (dataframe['datadate'] >= target_date - pd.Timedelta(window, 'M'))]
    if valid_dates.shape[0] < min_periods:
        return np.nan
    return calculate_forecast_from_extracted_factor(valid_dates)

def expanding_forecast_on_date(dataframe, target_date, min_periods):
    valid_dates = dataframe.loc[(dataframe['datadate'] <= target_date)]
    if valid_dates.shape[0] < min_periods:
        return np.nan
    return calculate_forecast_from_extracted_factor(valid_dates)

In [17]:
with Pool(4) as p:
    forecast_frame['Rolling Forecast'] = p.map(lambda t: rolling_forecast_on_date(forecast_frame, t, ROLL_WINDOW * 12, MIN_WINDOW * 12), date_sequence)

In [18]:
with Pool(4) as p:
    forecast_frame['Expanding Forecast'] = p.map(lambda t: expanding_forecast_on_date(forecast_frame, t, MIN_WINDOW * 12), date_sequence)

In [19]:
forecast_frame['Average So Far'] = forecast_frame['Next Month Return'].shift(1).expanding(MIN_WINDOW * 12).mean()

In [20]:
forecast_frame[['Rolling Forecast', 'Expanding Forecast', 'Average So Far', 'Next Month Return']].corr()

In [21]:
forecast_frame['Expanding Forecast Error'] = (forecast_frame['Next Month Return'] - forecast_frame['Expanding Forecast']) ** 2
forecast_frame['Rolling Forecast Error'] = (forecast_frame['Next Month Return'] - forecast_frame['Rolling Forecast']) ** 2
forecast_frame['Sample Mean Error'] = (forecast_frame['Next Month Return'] - forecast_frame['Average So Far']) ** 2

In [22]:
plot = TimePlot(title = 'Expected Returns vs Actual', legendLayout=LegendLayout.HORIZONTAL,\
                      legendPosition=LegendPosition(position=LegendPosition.Position.TOP),\
                    initWidth = 1000)
plot.add(Line(displayName = 'Expanding Forecast', \
              x = forecast_frame['datadate'],\
              y = forecast_frame['Expanding Forecast']))
plot.add(Line(displayName = 'Rolling Forecast', \
              x = forecast_frame['datadate'],\
              y = forecast_frame['Rolling Forecast']))

In [23]:
forecast_frame['Rolling 3PRF MSE'] = forecast_frame['Rolling Forecast Error'].rolling(MIN_WINDOW * 12).mean()
forecast_frame['Expanding 3PRF MSE'] = forecast_frame['Expanding Forecast Error'].rolling(MIN_WINDOW * 12).mean()
forecast_frame['Rolling Sample Mean MSE'] = forecast_frame['Sample Mean Error'].rolling(MIN_WINDOW * 12).mean()
forecast_frame['Expanding Sample Mean MSE'] = forecast_frame['Sample Mean Error'].rolling(MIN_WINDOW * 12).mean()

In [24]:
forecast_frame['OOS R Squared (Expanding)'] = 1 - forecast_frame['Expanding 3PRF MSE'] / forecast_frame['Expanding Sample Mean MSE']
forecast_frame['OOS R Squared (Rolling)'] = 1 - forecast_frame['Rolling 3PRF MSE'] / forecast_frame['Rolling Sample Mean MSE']

In [25]:
plot = TimePlot(title = 'OOS R2', legendLayout=LegendLayout.HORIZONTAL,\
                      legendPosition=LegendPosition(position=LegendPosition.Position.TOP),\
                    initWidth = 1000)
plot.add(Line(displayName = 'OOS Performance (Expanding)', \
              x = forecast_frame['datadate'],\
              y = forecast_frame['OOS R Squared (Expanding)']))
plot.add(Line(displayName = 'OOS Performance (Rolling)', \
              x = forecast_frame['datadate'],\
              y = forecast_frame['OOS R Squared (Rolling)']))

## Calculate a performance measure

In [50]:
forecast_frame['Buy and Hold'] = 1
FIXED_VAR = 0.2 ** 2

forecast_frame['Implied Risk Aversion'] = forecast_frame['Average So Far'] / FIXED_VAR
forecast_frame['Timing Strategy'] = forecast_frame['Expanding Forecast'] / forecast_frame['Implied Risk Aversion'] / FIXED_VAR
forecast_frame.loc[forecast_frame['Timing Strategy'] > 2, 'Timing Strategy'] = 2
forecast_frame.loc[forecast_frame['Timing Strategy'] < 0, 'Timing Strategy'] = 0

In [51]:
plot = TimePlot(title = 'Positions from Timing Strategy', legendLayout=LegendLayout.HORIZONTAL,\
                      legendPosition=LegendPosition(position=LegendPosition.Position.TOP),\
                    initWidth = 1000)
plot.add(Line(displayName = 'Timing Position', \
              x = forecast_frame['datadate'],\
              y = forecast_frame['Timing Strategy']))
plot.add(ConstantLine(y = 1))

In [52]:
forecast_frame['B&H Log Return'] = np.log(1 + forecast_frame['Buy and Hold'] * forecast_frame['Next Month Return'])
forecast_frame['3PRF Log Return'] = np.log(1 + forecast_frame['Timing Strategy'] * forecast_frame['Next Month Return'])

In [53]:
forecast_frame['B&H Cumulative Return'] = forecast_frame['B&H Log Return'].cumsum()
forecast_frame['3PRF Cumulative Return'] = forecast_frame['3PRF Log Return'].cumsum()
forecast_frame['B&H Cumulative Return'] = forecast_frame['B&H Cumulative Return'] - forecast_frame.loc[forecast_frame['datadate'] == '1970-01-31', 'B&H Cumulative Return']
forecast_frame['3PRF Cumulative Return'] = forecast_frame['3PRF Cumulative Return'] - forecast_frame.loc[forecast_frame['datadate'] == '1970-01-31', '3PRF Cumulative Return']
forecast_frame['Diff Return'] = forecast_frame['3PRF Cumulative Return'] - forecast_frame['B&H Cumulative Return']

In [54]:
plot = TimePlot(title = 'Cumulative Returns Since 1970', legendLayout=LegendLayout.HORIZONTAL,\
                      legendPosition=LegendPosition(position=LegendPosition.Position.TOP),\
                    initWidth = 1000)
plot.add(Line(displayName = 'Market Timing', \
              x = forecast_frame['datadate'],\
              y = forecast_frame['3PRF Cumulative Return']))
plot.add(Line(displayName = 'Buy and Hold', \
              x = forecast_frame['datadate'],\
              y = forecast_frame['B&H Cumulative Return']))

In [55]:
plot = TimePlot(title = 'Difference in Cumulative Returns Since 1970', legendLayout=LegendLayout.HORIZONTAL,\
                      legendPosition=LegendPosition(position=LegendPosition.Position.TOP),\
                    initWidth = 1000)
plot.add(Line(displayName = 'Market Timing - Buy and Hold', \
              x = forecast_frame['datadate'],\
              y = forecast_frame['Diff Return']))